In [1]:
from pytraj import io
import mdtraj as md

fname = "../../tests/data/nogit/remd/remd.000.nc"
topname = "../../tests/data/nogit/remd/myparm.parm7"

fa = io.load(fname, topname)
m_traj = md.load_netcdf(fname, top=topname)

print (fa)
print (m_traj)

<pytraj.Trajectory with 1000 frames: <Topology with 5634 mols, 5666 residues, 17443 atoms, 17452 bonds, PBC with box type = truncoct>>
           
<mdtraj.Trajectory with 1000 frames, 17443 atoms, 5666 residues, and unitcells>


### calculate distances for 1000 frames with 10*4 atompairs

In [2]:
import numpy as np
Nsize = 10**4
indices = np.random.randint(0, fa.size-1, size=Nsize*2).reshape(Nsize, 2)

print ("\n pytraj")
%timeit fa.calc_distance(indices)

print ("\n mdtraj: periodic=True, opt=True")
%timeit md.compute_distances(m_traj, indices)

print ("\n mdtraj: periodic=False, opt=True: fastest, using SSE")
%timeit md.compute_distances(m_traj, indices, periodic=False)

print ("\n mdtraj: periodic=False, opt=False (numpy version?)")
%timeit md.compute_distances(m_traj, indices, periodic=False, opt=False)

# mdtraj is much faster for single calculation (62 times faster, openmp?). 
# Not sure about including mask like :2-100@CB,CA ...


 pytraj
1 loops, best of 3: 901 ms per loop

 mdtraj: periodic=True, opt=True
1 loops, best of 3: 525 ms per loop

 mdtraj: periodic=False, opt=True: fastest, using SSE
1 loops, best of 3: 253 ms per loop

 mdtraj: periodic=False, opt=False (numpy version?)
1 loops, best of 3: 3.96 s per loop


### calculate distances for single frames with 10*4 atompairs

In [3]:
import numpy as np
Nsize = 10**4
indices = np.random.randint(0, fa.size-1, size=Nsize*2).reshape(Nsize, 2)
frame = fa[0]
m_frame = m_traj[0]

print ("\n pytraj")
%timeit frame.calc_distance(indices)

print ("\n mdtraj: periodic=True, opt=True")
%timeit md.compute_distances(m_frame, indices)

print ("\n mdtraj: periodic=False, opt=True: fastest, using SSE")
%timeit md.compute_distances(m_frame, indices, periodic=False)

print ("\n mdtraj: periodic=False, opt=False (numpy version?)")
%timeit md.compute_distances(m_frame, indices, periodic=False, opt=False)

# mdtraj is much faster for single calculation (62 times faster, openmp?). 
# Not sure about including mask like :2-100@CB,CA ...


 pytraj
1000 loops, best of 3: 774 µs per loop

 mdtraj: periodic=True, opt=True
1000 loops, best of 3: 908 µs per loop

 mdtraj: periodic=False, opt=True: fastest, using SSE
1000 loops, best of 3: 339 µs per loop

 mdtraj: periodic=False, opt=False (numpy version?)
100 loops, best of 3: 2.14 ms per loop


### calculate distances for 1000 frames with 10*4 atompairs, using `iterload`

In [1]:
import numpy as np
import mdtraj as md
import pytraj.io as io


def calc_pytraj_iterload_frame_iter():
    Nsize = 10**4
    indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)
    fname = "../../tests/data/nogit/remd/remd.000.nc"
    topname = "../../tests/data/nogit/remd/myparm.parm7"
    fa = io.iterload(fname, topname)
    fa.calc_distance(indices)
    
def calc_pytraj_iterload_frame_iter_openmp():
    Nsize = 10**4
    indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)
    fname = "../../tests/data/nogit/remd/remd.000.nc"
    topname = "../../tests/data/nogit/remd/myparm.parm7"
    fa = io.iterload(fname, topname)
    for frame in fa:
        frame.calc_distance(indices, parallel=True)

def calc_pytraj_iterload_chunk_iter():
    Nsize = 10**4
    indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)
    fname = "../../tests/data/nogit/remd/remd.000.nc"
    topname = "../../tests/data/nogit/remd/myparm.parm7"
    fa = io.iterload(fname, topname)
    for chunk in fa.chunk_iter(chunk=100):
        chunk.calc_distance(indices)
    
def calc_mdtraj_iterload():
    Nsize = 10**4
    indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)
    fname = "../../tests/data/nogit/remd/remd.000.nc"
    topname = "../../tests/data/nogit/remd/myparm.parm7"
    for chunk in md.iterload(fname, top=topname):
        md.compute_distances(chunk, indices, periodic=False, opt=True)

print ("\n pytraj: frame_iter")
%timeit calc_pytraj_iterload_frame_iter()

print ("\n pytraj: frame_iter: openmp") # poor scaling.
%timeit calc_pytraj_iterload_frame_iter_openmp()

print ("\n pytraj: chunk_iter")
%timeit calc_pytraj_iterload_chunk_iter()

print ("\n mdtraj: periodic=False, opt=True: fastest, using SSE")
%timeit calc_mdtraj_iterload()


 pytraj: frame_iter
1 loops, best of 3: 2.15 s per loop

 pytraj: frame_iter: openmp
1 loops, best of 3: 1.23 s per loop

 pytraj: chunk_iter
1 loops, best of 3: 2.86 s per loop

 mdtraj: periodic=False, opt=True: fastest, using SSE
1 loops, best of 3: 3.45 s per loop
